In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib as plt

In [2]:
from NRI.experiments.config import ExperimentConfig


experiment_config = ExperimentConfig(
    obs_len=12,
    pred_len=18,
    encoder_loss_weight=1.0,
    decoder_loss_weight=1.0,
    signal_loss_weight=1.0,
    n_epoch=20,
    batch_size=64,
    checkpoint_prefix='dnri_18_NoTraff',
    checkpoint_interval=10,
    use_cuda=True,
    mask_traffic_signal=True
)

In [3]:
from SinD.config import get_dataset_path
from SinD.dataset.io import get_dataset_records
from NRI.dataset.utils import split_dataset

dataset_path = get_dataset_path()
dataset_files = get_dataset_records(dataset_path)
train_files, valid_files, test_files = split_dataset(dataset_files, 0.7, 0.2)

load datasets

In [4]:
from NRI.dataset import SignalizedIntersectionDatasetForNRI, SignalizedIntersectionDatasetConfig

dataset_config = SignalizedIntersectionDatasetConfig(
    obs_len=experiment_config.obs_len,
    pred_len=experiment_config.pred_len,
    stride=15,
    encode_traffic_signals=True,
    padding_value=0.0
)

train_set = SignalizedIntersectionDatasetForNRI(dataset_config)
train_set.load_records(dataset_path, train_files, verbose=True)

valid_set = SignalizedIntersectionDatasetForNRI(dataset_config)
valid_set.load_records(dataset_path, valid_files, verbose=True)


load_records: 100%|██████████| 4/4 [00:40<00:00, 10.19s/it]


In [5]:
from NRI.models import DynamicNeuralRelationalInference
from NRI.experiments.main import train

model = DynamicNeuralRelationalInference(
    hid_dim=64,
    n_edges=4,
    dgvae=False
)

if experiment_config.use_cuda:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters())

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

train(
    model,
    train_set=train_set,
    valid_set=valid_set,
    config=experiment_config,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler
)

[train] epoch 0:   0%|          | 0/172 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

Overall Loss: 33.77419683545136
Encoder KL Loss: 0.018318692989089686
Decoder NLL Loss: 33.75587829323702
Signal Cross-Entropy Loss: 1.7816878862159162


[valid] epoch 0:   0%|          | 0/48 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: Th

Signal Prediction Accuracy: 0.0
Final Displacement Error: 4.775298660000165
Average Displacement Error: 2.228523169954618


[train] epoch 1:   0%|          | 0/172 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

...

Signal Prediction Accuracy: 0.0
Final Displacement Error: 4.911520704627037
Average Displacement Error: 2.1511467744906745


[train] epoch 19:   0%|          | 0/172 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: 

Overall Loss: -4.940957611383396
Encoder KL Loss: 0.06015496475752011
Decoder NLL Loss: -5.001112581165724
Signal Cross-Entropy Loss: 1.7816367897876466


[valid] epoch 19:   0%|          | 0/48 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

Signal Prediction Accuracy: 0.0
Final Displacement Error: 4.926220695177714
Average Displacement Error: 2.172109251221021


In [1]:
import os
import torch
from torch.utils.data import DataLoader

from NRI.dataset import SignalizedIntersectionDatasetForNRI
from NRI.models import DynamicNeuralRelationalInference
from NRI.experiments.config import ExperimentConfig
from NRI.experiments.test import generate_result
from SinD.config import get_dataset_path
from SinD.dataset.io import get_dataset_records
from NRI.dataset.utils import split_dataset
from NRI.dataset import SignalizedIntersectionDatasetForNRI, SignalizedIntersectionDatasetConfig

experiment_config = ExperimentConfig(
    obs_len=12,
    pred_len=12,
    encoder_loss_weight=1.0,
    decoder_loss_weight=1.0,
    signal_loss_weight=1.0,
    n_epoch=20,
    batch_size=64,
    checkpoint_prefix='dnri_18_NoTraff',
    checkpoint_interval=10,
    use_cuda=True,
    mask_traffic_signal=True
)

# Load the dataset
dataset_path = get_dataset_path()
dataset_files = get_dataset_records(dataset_path)
_, _, test_files = split_dataset(dataset_files, 0.7, 0.2)

dataset_config = SignalizedIntersectionDatasetConfig(
    obs_len=experiment_config.obs_len,
    pred_len=experiment_config.pred_len,
    stride=15,
    encode_traffic_signals=True,
    padding_value=0.0,
    include_incomplete_trajectories=False
)

test_set = SignalizedIntersectionDatasetForNRI(dataset_config)
test_set.load_records(dataset_path, test_files, verbose=True)

model = DynamicNeuralRelationalInference(
    hid_dim=64,
    n_edges=4,
    dgvae=False
)

# Load the saved model and configuration
checkpoint = torch.load('../checkpoints/dnri_18_NoTraff_best.pt')
model.load_state_dict(checkpoint['params'])
test_stats = generate_result(model, test_set, experiment_config)

# Report the results
test_stats.report()

[test] generating result:   0%|          | 0/34 [00:00<?, ?it/s]c:\Users\chinp\source\AnythingVisionaries\venv\Lib\site-packages\torch\nested\__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ..\aten\src\ATen\NestedTensorImpl.cpp:180.)
  return _nested.nested_tensor(
[test] generating result: 100%|██████████| 34/34 [00:06<00:00,  5.39it/s]

=== Overall Result ===
Signal Prediction Accuracy: 0.0
Overall Final Displacement Error: 2.8654088973999023
Overall Average Displacement Error: 1.1738905906677246
=== Displacement Error of Class `pedestrian` ===
FDE: 5.389487702640204
ADE: 2.7717833351974304
=== Displacement Error of Class `car` ===
FDE: 2.6083696736347872
ADE: 0.7664632424754578
=== Displacement Error of Class `truck` ===
FDE: 20.44562187887007
ADE: 6.0214136608185305
=== Displacement Error of Class `bus` ===
FDE: 0.7366150116608537
ADE: 0.262057336327435
=== Displacement Error of Class `motorcycle` ===
FDE: 3.0535265687537145
ADE: 1.1208451778813462
=== Displacement Error of Class `tricycle` ===
FDE: 2.2579938850605688
ADE: 0.977609463123894
=== Displacement Error of Class `bicycle` ===
FDE: 4.7598493628458005
ADE: 2.132578789074484
=== Overall Result ===
Signal Prediction Accuracy: 0.0
Overall Final Displacement Error: 2.8654088973999023
Overall Average Displacement Error: 1.1738905906677246
=== Displacement Error o